# check if freestyle output will be white bg or not after remove alpha channel

In [ ]:

import os
os.environ["PYOPENGL_PLATFORM"] = "osmesa"

import pyrender
import trimesh
import numpy as np
from PIL import Image
import os


import trimesh
import pyrender
import numpy as np
from PIL import Image

scene_t = trimesh.load("/home/athiwat/progressive_img2sketch/resources/LOD_for_icp/1/lod2.obj")  # may return Trimesh or Scene

pr_scene = None
if isinstance(scene_t, trimesh.Scene):
    pr_scene = pyrender.Scene.from_trimesh_scene(scene_t)
else:
    mesh = pyrender.Mesh.from_trimesh(scene_t)
    pr_scene = pyrender.Scene()
    pr_scene.add(mesh)

# Add camera
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=640/480)
cam_pose = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, -1],
    [0, 0, 1, 2.5],
    [0, 0, 0, 1],
])
pr_scene.add(camera, pose=cam_pose)

# Add light
light = pyrender.SpotLight(color=np.ones(3), intensity=3.0)
pr_scene.add(light, pose=cam_pose)

r = pyrender.OffscreenRenderer(640, 480)
color, depth = r.render(pr_scene)
Image.fromarray(color.astype(np.uint8)).save("offscreen_render.png")
r.delete()


In [ ]:
import os
import cv2
import numpy as np

input_image = "/home/athiwat/progressive_img2sketch/resources/LOD_orbit_freestyles_tmp/39/lod3/lod3_az000_el00.png"
output_image = "/home/athiwat/progressive_img2sketch/test/test_sanity.png"

# Load image with alpha
image_rgba = cv2.imread(input_image, cv2.IMREAD_UNCHANGED)
print(f"Image shape: {image_rgba.shape}", "Image dtype:", image_rgba.dtype)

cv2.imwrite("/home/athiwat/progressive_img2sketch/test/before.png", image_rgba)

# Check input has 4 channels
if image_rgba is None or image_rgba.shape[2] != 4:
    raise ValueError("Input image must be RGBA with 4 channels")

# Separate RGBA
rgb = image_rgba[:, :, :3].astype(np.float32)
alpha = image_rgba[:, :, 3].astype(np.float32) / 255.0  # Normalize to [0,1]

# Define background gray value (0 = black, 255 = white, e.g. 200 = light gray)
bg_gray_value = 255
bg_gray = np.ones_like(rgb) * bg_gray_value

# Alpha blending: result = rgb * alpha + gray * (1 - alpha)
blended_rgb = (rgb * alpha[..., None] + bg_gray * (1 - alpha[..., None])).astype(np.uint8)

# Optionally print stats
print(f"Blended image shape: {blended_rgb.shape}", "Blended image dtype:", blended_rgb.dtype)
print("Image average color:", np.mean(blended_rgb, axis=(0, 1)))

# Save the final result
cv2.imwrite(output_image, blended_rgb)
print(f"Converted image saved to {output_image}")
